In [1]:
import requests
import datetime as dt
import pandas as pd

## Задание 1. Шутки

In [2]:
response = requests.get('https://official-joke-api.appspot.com/jokes/ten')

In [3]:
# выполняется около минуты, 350 шуток
jokes_types = set()
for k in range(25): 
    response = requests.get('https://official-joke-api.appspot.com/jokes/ten')
    for i in range(len(response.json())): 
        jokes_types.add(response.json()[i]['type'])

In [4]:
jokes_types

{'dad', 'general', 'knock-knock', 'programming'}

In [5]:
for joke_type in jokes_types: 
    response = requests.get(f'https://official-joke-api.appspot.com/jokes/{joke_type}/ten')
    print("Категория:", joke_type)
    for i in range(len(response.json())): 
        print(response.json()[i]['setup'] + '\n' + response.json()[i]['punchline'])
        print()
    print()    
    print()

Категория: dad
Why do fathers take an extra pair of socks when they go golfing?
In case they get a hole in one!

Why don't eggs tell jokes?
Because they would crack each other up.



Категория: general
What did one nut say as he chased another nut?
 I'm a cashew!

What do you call corn that joins the army?
Kernel.

What is the hardest part about sky diving?
The ground.

Who did the wizard marry?
His ghoul-friend

How do you get two whales in a car?
Start in England and drive West.

Is the pool safe for diving?
It deep ends.

What does a female snake use for support?
A co-Bra!

Why did the rooster cross the road?
He heard that the chickens at KFC were pretty hot.

What did one plate say to the other plate?
Dinner is on me!

What did celery say when he broke up with his girlfriend?
She wasn't right for me, so I really don't carrot all.



Категория: programming
Which song would an exception sing?
Can't catch me - Avicii

Knock-knock.
A race condition. Who is there?

Why did the programme

## Задание 2. Валюты

In [6]:
# 100 USD -> RUB
response = requests.get('https://open.er-api.com/v6/latest/USD')
if response:
    rate = response.json()['rates']['RUB']
    print(f'100 USD is {round(rate * 100, 2)} RUB')
else:
    print(f'An error has occurred. Status code: {response.status_code}')

100 USD is 7094.88 RUB


In [7]:
# 100 RUB -> EUR
response = requests.get('https://open.er-api.com/v6/latest/RUB')
if response:
    rate = response.json()['rates']['EUR']
    print(f'100 RUB is {round(rate * 100, 2)} EUR')
else:
    print(f'An error has occurred. Status code: {response.status_code}')    

100 RUB is 1.31 EUR


In [8]:
# 100 EUR -> USD
response = requests.get('https://open.er-api.com/v6/latest/EUR')
if response:
    rate = response.json()['rates']['USD']
    print(f'100 EUR is {round(rate * 100, 2)} USD')
else:
    print(f'An error has occurred. Status code: {response.status_code}')    

100 EUR is 107.32 USD


### Задание 3. Публичные API

In [9]:
response = requests.get('https://api.publicapis.org/entries')

In [10]:
auth_counter = {}
githab_count = 0
category_counter = {}

for i in range(len(response.json()['entries'])): 
    if response.json()['entries'][i]['Auth'] in auth_counter.keys(): 
        auth_counter[response.json()['entries'][i]['Auth']] += 1    
    else: 
        auth_counter[response.json()['entries'][i]['Auth']] = 1
        
    if response.json()['entries'][i]['Link'].find('github.io') != -1: 
        githab_count += 1
        
    if response.json()['entries'][i]['Category'] in category_counter.keys(): 
        category_counter[response.json()['entries'][i]['Category']] += 1    
    else: 
        category_counter[response.json()['entries'][i]['Category']] = 1

In [11]:
auth_df = pd.DataFrame(auth_counter.values(), index=auth_counter.keys(), columns=['count'])
auth_df['share_%'] = round(auth_df['count'] / auth_df['count'].sum() * 100, 2)

In [12]:
auth_df

,count,share_%
apiKey,600,42.11
,669,46.95
OAuth,149,10.46
X-Mashape-Key,6,0.42
User-Agent,1,0.07


In [13]:
print(f'На гитхабе развернуто {githab_count} API')

На гитхабе развернуто 24 API


In [14]:
category_counter

{'Animals': 27,
 'Anime': 19,
 'Anti-Malware': 15,
 'Art & Design': 20,
 'Authentication & Authorization': 7,
 'Blockchain': 11,
 'Books': 23,
 'Business': 23,
 'Calendar': 16,
 'Cloud Storage & File Sharing': 19,
 'Continuous Integration': 6,
 'Cryptocurrency': 64,
 'Currency Exchange': 17,
 'Data Validation': 7,
 'Development': 120,
 'Dictionaries': 13,
 'Documents & Productivity': 28,
 'Email': 17,
 'Entertainment': 10,
 'Environment': 17,
 'Events': 3,
 'Finance': 45,
 'Food & Drink': 24,
 'Games & Comics': 96,
 'Geocoding': 86,
 'Government': 86,
 'Health': 31,
 'Jobs': 17,
 'Machine Learning': 22,
 'Music': 33,
 'News': 19,
 'Open Data': 35,
 'Open Source Projects': 9,
 'Patent': 4,
 'Personality': 23,
 'Phone': 5,
 'Photography': 28,
 'Programming': 5,
 'Science & Math': 33,
 'Security': 38,
 'Shopping': 14,
 'Social': 40,
 'Sports & Fitness': 33,
 'Test Data': 25,
 'Text Analysis': 15,
 'Tracking': 9,
 'Transportation': 69,
 'URL Shorteners': 19,
 'Vehicle': 6,
 'Video': 43,
 '

## 4. Календарь соревнований по датам на ближайшие 15 дней

In [15]:
response = requests.get('https://kontests.net/api/v1/sites')
today = dt.datetime.today()
next_15_days = today + dt.timedelta(days=15)
calendar_df = pd.DataFrame(columns=['name', 'url', 'start_datetime', 'end_datetime'])

In [16]:
for site in response.json(): 
    if requests.get(f'https://kontests.net/api/v1/{site[1]}'): 
        for contest in requests.get(f'https://kontests.net/api/v1/{site[1]}').json(): 
            if 'Z' in contest['start_time']: 
                start_dt = dt.datetime.fromisoformat(contest['start_time'].replace('Z', '')) 
                end_dt = dt.datetime.fromisoformat(contest['end_time'].replace('Z', '')) 

            else: 
                datetime_format = '%Y-%m-%d %H:%M:%S %Z'
                start_dt = dt.datetime.strptime(contest['start_time'], datetime_format) 
                end_dt = dt.datetime.strptime(contest['end_time'], datetime_format)
            if start_dt <= next_15_days and end_dt >= today: 
                new_df = pd.DataFrame({'name' : contest['name'],
                                   'url' : contest['url'],           
                                   'start_datetime': start_dt,
                                   'end_datetime' : end_dt}, index=[0])
                calendar_df = pd.concat([calendar_df, new_df])
                
calendar_df = calendar_df.reset_index(drop=True)                

In [17]:
calendar_df

,name,url,start_datetime,end_datetime
0,Codeforces Round #851 (Div. 2),https://codeforces.com/contestRegistration/1788,2023-02-09 14:35:00,2023-02-09 16:35:00
1,Codeforces Round #852 (Div. 2),https://codeforces.com/contestRegistration/1793,2023-02-12 08:35:00,2023-02-12 10:35:00
2,Educational Codeforces Round 143 (Rated for Di...,https://codeforces.com/contestRegistration/1795,2023-02-13 14:35:00,2023-02-13 16:35:00
3,"SWERC 2022-2023 - Online Mirror (Unrated, ICPC...",https://codeforces.com/contestRegistration/1776,2023-02-19 11:05:00,2023-02-19 16:05:00
4,"Ｓｋｙ Inc, Programming Contest 2023（AtCoder Begi...",https://atcoder.jp/contests/abc289,2023-02-11 12:00:00,2023-02-11 13:40:00
5,AtCoder Grand Contest 061,https://atcoder.jp/contests/agc061,2023-02-12 12:00:00,2023-02-12 15:00:00
6,RECRUIT Nihonbashi Half Marathon 2023 Winter（A...,https://atcoder.jp/contests/ahc018,2023-02-18 03:00:00,2023-02-26 10:00:00
7,Toyota Programming Contest 2023 Spring Qual B（...,https://atcoder.jp/contests/abc290,2023-02-19 12:00:00,2023-02-19 13:40:00
8,Starters 77,https://www.codechef.com/START77,2023-02-15 14:30:00,2023-02-15 17:30:00
9,February Cook-Off 2023 (Rated for All),https://www.codechef.com/COOK146,2023-02-08 14:30:00,2023-02-08 17:00:00


## Задача 5. Посчитать зарплату работников за весну 2022 года

In [18]:
input_data = \
'''Ivan Ivanov,1000 USD
Alex Random,1000 EUR
Athur Smith,1000 RUB
Masha Noname,130 RUB
Krolik Rodger,5 RUB'''

In [19]:
#таблица сотрудников
df = pd.DataFrame()
for row in input_data.splitlines(): 
    employee = pd.DataFrame({'name' : row.split(',')[0], 'hour_rate' : row.split(',')[1]}, index=[0])
    df = pd.concat([df, employee])
    
df['currency'] = df['hour_rate'].apply(lambda x: x.split(' ')[1])
df['hour_rate'] = df['hour_rate'].apply(lambda x: int(x.split(' ')[0]))
df = df.reset_index(drop=True)    

In [20]:
working_hours = {3 : 0, 4 : 0, 5 : 0}
total_spring_working_hours = 0

In [21]:
for month_num in (working_hours.keys()): 
    response = requests.get(f'https://isdayoff.ru/api/getdata?year=2022&month={month_num}')
    if response.status_code == 200:
        zeros = str(response.content).count('0')
        twos = str(response.content).count('2')
        fours = str(response.content).count('4')
        working_hours[month_num] = (zeros + fours) * 8 + twos * 7
        total_spring_working_hours += working_hours[month_num]
    else:
        print(f'An error has occurred. Month: {month_num}. Status code: {response.status_code}')

In [22]:
working_hours

{3: 176, 4: 168, 5: 144}

In [23]:
total_spring_working_hours

488

In [24]:
df['spring_salary'] = df['hour_rate'].apply(lambda x: x * total_spring_working_hours)

In [25]:
df

,name,hour_rate,currency,spring_salary
0,Ivan Ivanov,1000,USD,488000
1,Alex Random,1000,EUR,488000
2,Athur Smith,1000,RUB,488000
3,Masha Noname,130,RUB,63440
4,Krolik Rodger,5,RUB,2440
